# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
# Your solution goes here

# Import necessary libraries
import re
import time
from typing import List, Dict, Any, Optional

import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [6]:
BASE_URL = "http://books.toscrape.com/"
CATALOGUE_URL = urljoin(BASE_URL, "catalogue/")

RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5,
}

In [7]:
# FUnction definitions now all in one cell for easier execution

def _get_soup(url: str, session: requests.Session, timeout: int = 30) -> BeautifulSoup:
    resp = session.get(url, timeout=timeout)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")


def _parse_price_gbp(price_text: str) -> Optional[float]:
    """
    Examples seen on site:
      '£51.77'
      '£20.00'
    """
    if not price_text:
        return None
    # keep digits + dot
    m = re.search(r"(\d+\.\d+)", price_text)
    return float(m.group(1)) if m else None


def _parse_rating_from_class(tag) -> Optional[int]:
    """
    Rating on listing and product page is like:
      <p class="star-rating Three">...</p>
    We grab the word class (One..Five) and map it.
    """
    if tag is None:
        return None
    classes = tag.get("class", [])
    # classes = ["star-rating", "Three"]
    for c in classes:
        if c in RATING_MAP:
            return RATING_MAP[c]
    return None


def _extract_product_table(soup: BeautifulSoup) -> Dict[str, str]:
    """
    On product page there's a table with rows like:
      UPC, Product Type, Price (excl. tax), Price (incl. tax), Tax, Availability, Number of reviews
    """
    out = {}
    table = soup.select_one("table.table.table-striped")
    if not table:
        return out
    for row in table.select("tr"):
        th = row.find("th")
        td = row.find("td")
        if th and td:
            out[th.get_text(strip=True)] = td.get_text(strip=True)
    return out


def _extract_breadcrumb_genre(soup: BeautifulSoup) -> Optional[str]:
    """
    Breadcrumb:
      Home > Books > Travel > It's Only the Himalayas
    Genre = the 3rd breadcrumb item typically (index 2).
    """
    crumbs = soup.select("ul.breadcrumb li a")
    # Anchors: Home, Books, Genre
    if len(crumbs) >= 3:
        return crumbs[2].get_text(strip=True)
    return None


def _extract_description(soup: BeautifulSoup) -> Optional[str]:
    """
    Description is usually the <p> following the div#product_description header.
    Sometimes it's missing.
    """
    header = soup.select_one("#product_description")
    if not header:
        return None
    # find the next <p> after the header
    p = header.find_next("p")
    if not p:
        return None
    txt = p.get_text(" ", strip=True)
    return txt if txt else None


def scrape_books(min_rating: float = 4.0, max_price: float = 20.0, polite_delay_s: float = 0.2) -> pd.DataFrame:
    """
    Scrape Books to Scrape and return a DataFrame with:
      UPC, Title, Price (£), Rating, Genre, Availability, Description

    Filters:
      Rating >= min_rating
      Price  <= max_price
    """
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; IronhackLabScraper/1.0; +https://books.toscrape.com/)"
    })

    results: List[Dict[str, Any]] = []

    page_url = urljoin(CATALOGUE_URL, "page-1.html")

    while True:
        soup = _get_soup(page_url, session)

        # Each book card
        articles = soup.select("article.product_pod")
        if not articles:
            break

        for art in articles:
            title_tag = art.select_one("h3 a")
            if not title_tag:
                continue

            title = title_tag.get("title", "").strip()
            rel_href = title_tag.get("href", "")
            # Listing pages are under /catalogue/ already
            book_url = urljoin(page_url, rel_href)

            price_tag = art.select_one(".price_color")
            price = _parse_price_gbp(price_tag.get_text(strip=True) if price_tag else "")

            rating_tag = art.select_one("p.star-rating")
            rating = _parse_rating_from_class(rating_tag)

            # quick filter before hitting detail page
            if price is None or rating is None:
                continue
            if rating < min_rating or price > max_price:
                continue

            # detail page for UPC, genre, description, availability
            time.sleep(polite_delay_s)
            detail = _get_soup(book_url, session)

            product_table = _extract_product_table(detail)
            upc = product_table.get("UPC")
            availability = product_table.get("Availability")

            genre = _extract_breadcrumb_genre(detail)
            description = _extract_description(detail)

            # In case some pages also have price incl tax; we stick to listing price requested
            results.append({
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description,
            })

        # Pagination: look for 'li.next a'
        next_link = soup.select_one("li.next a")
        if not next_link:
            break

        next_href = next_link.get("href")
        page_url = urljoin(page_url, next_href)

    df = pd.DataFrame(results, columns=["UPC", "Title", "Price (£)", "Rating", "Genre", "Availability", "Description"])
    return df

In [ ]:
books_df = scrape_books(min_rating=4.0, max_price=20.0)
books_df.shape

# Removed the S from http BASE URL = "http://books.toscrape.com/", which allowed the code to run without SSL errors.
# Performed due to SSL certificate verification issues on the original site, will not repeat this aproach for real sites


(75, 7)